In [1]:
import pymapee as pymap
import geemap as emap
import ee
pymap.initialize_ee()

## 1. Interpolating cloud-related pixels

In [2]:
# Area of interest
aoi=ee.FeatureCollection("projects/tuyenhavan/assets/Julian_Province")
# MODIS LAI Terra (example)
terra=ee.ImageCollection("MODIS/061/MOD15A2H")
# Masking cloud-related pixels
terra_mask=pymap.modis_cloud_mask(terra,5,7,"FparLai_QC",1)

lai_clip=terra_mask.map(lambda img: img.clip(aoi))

In [3]:
# Interpolating missing values
lai_test=pymap.gee_linear_interpolate_nan(lai_clip, 60)
lai_test.size().getInfo()

1072

## 2. Split the study area into chunks

In [4]:
aoi=ee.FeatureCollection("projects/tuyenhavan/assets/Vietnam_PhD")

chunks=pymap.chunk_maker(aoi, ncols=10, nrows=10)
# Visualize the chunks
Map=emap.Map()
Map.centerObject(aoi, 5)
Map.addLayer(chunks, {}, "Chunks")
Map.addLayer(aoi, {}, "VN Map")
Map

Map(center=[16.547396784175472, 106.35557847623937], controls=(WidgetControl(options=['position', 'transparent…

We can loop over each tile and do computation.

In [5]:
mlist=[]

for feat in chunks.getInfo()["features"]:
    tem_feat=ee.Feature(feat)
    mlist.append(tem_feat)
new_feat_col=ee.FeatureCollection(mlist)

In [6]:
# Visualize the last tile
Map=emap.Map()
Map.centerObject(tem_feat)
Map.addLayer(tem_feat)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…